In [ ]:
!pip install -U ipykernel
!pip install -q streamlit
!pip install pyngrok

     |████████████████████████████████| 122kB 2.8MB/s 
ERROR: Operation cancelled by user
     |████████████████████████████████| 7.2MB 2.9MB/s 
     |████████████████████████████████| 102kB 10.0MB/s 
     |████████████████████████████████| 4.4MB 44.8MB/s 
     |████████████████████████████████| 112kB 51.4MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.4 which is incompatible.


In [1]:
!pip install PyYAML
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 243 (delta 0), reused 1 (delta 0), pack-reused 239
Receiving objects: 100% (243/243), 71.46 MiB | 34.15 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [ ]:
cd first-order-model

/content/first-order-model


In [ ]:
import sys
sys.path.append('first-order-model')

In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
from google.colab import files
upload1 = files.upload()

In [ ]:
upload2 = files.upload()

Saving 00.mp4 to 00.mp4


In [ ]:
!pwd

/content


In [2]:
cd first-order-model

/content/first-order-model


In [ ]:
!ngrok authtoken 1gBC1BizajDqpzJUZq3DFx0MDLJ_3oikNo2K7U6Si2MX9mWCm

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import streamlit as st
#st.set_option('deprecation.showfileUploaderEncoding', False)
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

In [ ]:
import sys
sys.path.insert(0, 'first-order-model')

In [ ]:
from demo import load_checkpoints

In [ ]:
#! ngrok authtoken 1gBC1BizajDqpzJUZq3DFx0MDLJ_3oikNo2K7U6Si2MX9mWCm

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
uploaded4 = files.upload()

Saving vox-256.yaml to vox-256.yaml


In [ ]:
cd ..

/content


In [ ]:
%%writefile score.py
import streamlit as st

#st.set_option('deprecation.showfileUploaderEncoding', False)
import imageio
import numpy as np
import sys
sys.path.insert(0, 'first-order-model')
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize

import warnings
warnings.filterwarnings("ignore")
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte
import io
import os
import base64

def get_binary_file_downloader_html(bin_file, file_label='File'):
    with open(bin_file, 'rb') as f:
        data = f.read()
    bin_str = base64.b64encode(data).decode()
    href = f'<a href="data:application/octet-stream;base64,{bin_str}" download="generated.mp4">Video Download Link</a>'
    return href

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

st.title("Deepfake Generator")
st.subheader("Produce synthetic videos with AI!")

st.header("Upload Image:")
st.subheader("Synthetic video will be produced from this image")
image_file = st.text_input("Enter Image path/name:")
st.text('')

st.header("Upload Video File:")
st.subheader("The movements of the person in this video will be used for producing the synthetic video.")
video_file = st.text_input("Enter Video path/name:")

if st.button('Generate Video!'):
    if image_file is None or video_file is None:
        st.markdown("# Please upload both Image and Video.")
    else:
        source_image = imageio.imread('02.png')
        driving_video = imageio.mimread('00.mp4')

        with st.spinner('Processing the video...'):
            source_image = resize(source_image, (256,256))[..., :3]
            driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]
            
            
        
            generator, kp_detector = load_checkpoints(config_path='vox-256.yaml', checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')
            predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
            #test = imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
            imageio.mimsave('generated.mp4', [img_as_ubyte(frame) for frame in predictions])
            st.video('generated.mp4')
            #st.video(test)
            
            st.markdown(get_binary_file_downloader_html('generated.mp4', 'Picture'), unsafe_allow_html=True)
        st.balloons()

Overwriting score.py


In [ ]:
!nohup streamlit run score.py &

nohup: appending output to 'nohup.out'


In [ ]:
from pyngrok import ngrok

url = ngrok.connect(port=8501)
url

2020-08-18 09:12:33.243 INFO    pyngrok.process: ngrok process starting: 508
2020-08-18 09:12:33.262 INFO    pyngrok.process: t=2020-08-18T09:12:33+0000 lvl=info msg="no configuration paths supplied"

2020-08-18 09:12:33.264 INFO    pyngrok.process: t=2020-08-18T09:12:33+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

2020-08-18 09:12:33.265 INFO    pyngrok.process: t=2020-08-18T09:12:33+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

2020-08-18 09:12:33.268 INFO    pyngrok.process: t=2020-08-18T09:12:33+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040

2020-08-18 09:12:33.439 INFO    pyngrok.process: t=2020-08-18T09:12:33+0000 lvl=info msg="tunnel session started" obj=tunnels.session

2020-08-18 09:12:33.443 INFO    pyngrok.process: t=2020-08-18T09:12:33+0000 lvl=info msg="client session established" obj=csess id=edff57869421

2020-08-18 09:12:33.450 INFO    pyngrok.process: ngrok process has s

'http://bc93118ffb22.ngrok.io'

2020-08-18 09:12:33.536 INFO    pyngrok.process: t=2020-08-18T09:12:33+0000 lvl=info msg=end pg=/api/tunnels id=01649b82cca76da0 status=201 dur=73.627834ms



In [ ]:
ngrok.disconnect(public_url="https://16a89003891e.ngrok.io/")

2020-08-18 00:15:36.223 INFO    pyngrok.process: t=2020-08-18T00:15:36+0000 lvl=info msg=start pg=/api/tunnels id=32c39543e1a74f03

2020-08-18 00:15:36.224 INFO    pyngrok.process: t=2020-08-18T00:15:36+0000 lvl=info msg=end pg=/api/tunnels id=32c39543e1a74f03 status=200 dur=337.71µs



In [ ]:
! git clone https://github.com/streamlit/demo-face-gan.git

Cloning into 'demo-face-gan'...
remote: Enumerating objects: 172, done.
remote: Total 172 (delta 0), reused 0 (delta 0), pack-reused 172
Receiving objects: 100% (172/172), 32.11 MiB | 30.79 MiB/s, done.
Resolving deltas: 100% (78/78), done.


In [ ]:
%%writefile score2.py
%cd demo-face-gan
import numpy as np
import os
import pickle
import streamlit as st
import sys
import tensorflow as tf
import urllib
from PIL import Image
import base64
import io

sys.path.append('tl_gan')
sys.path.append('pg_gan')
import feature_axis
import tfutil
import tfutil_cpu

# This should not be hashed by Streamlit when using st.cache.
TL_GAN_HASH_FUNCS = {
    tf.Session : id
}

def get_image_download_link(img):
    buffered = io.BytesIO()
    img.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode()
    href = f'<a href="data:file/jpg;base64,{img_str}" download="Your_File.jpg">Download result</a>'
    return href

def main():
    st.title("Streamlit Face-GAN Demo")
    # Download all data files if they aren't already in the working directory.
    for filename in EXTERNAL_DEPENDENCIES.keys():
        download_file(filename)

    # Read in models from the data files.
    tl_gan_model, feature_names = load_tl_gan_model()
    session, pg_gan_model = load_pg_gan_model()

    st.sidebar.title('Features')
    seed = 27834096
    # If the user doesn't want to select which features to control, these will be used.
    default_control_features = ['Young','Smiling','Male']
    if st.sidebar.checkbox('Show advanced options'):
        # Randomly initialize feature values. 
        features = get_random_features(feature_names, seed)
        # Let the user pick which features to control with sliders.
        control_features = st.sidebar.multiselect( 'Control which features?',
            sorted(features), default_control_features)
    else:
        features = get_random_features(feature_names, seed)
        # Don't let the user pick feature values to control.
        control_features = default_control_features
    
    # Insert user-controlled values from sliders into the feature vector.
    for feature in control_features:
        features[feature] = st.sidebar.slider(feature, 0, 100, 50, 5)

    # Generate a new image from this feature vector (or retrieve it from the cache).
    with session.as_default():
        image_out = generate_image(session, pg_gan_model, tl_gan_model,
                features, feature_names)

    st.image(image_out, use_column_width=False, width=550)
    result = Image.fromarray(image_out)

    if st.button('Download Image'):
        st.markdown(get_image_download_link(result), unsafe_allow_html=True)
        st.balloons()

def download_file(file_path):
    # Don't download the file twice. (If possible, verify the download using the file length.)
    if os.path.exists(file_path):
        if "size" not in EXTERNAL_DEPENDENCIES[file_path]:
            return
        elif os.path.getsize(file_path) == EXTERNAL_DEPENDENCIES[file_path]["size"]:
            return

    # These are handles to two visual elements to animate.
    weights_warning, progress_bar = None, None
    try:
        weights_warning = st.warning("Downloading %s..." % file_path)
        progress_bar = st.progress(0)
        with open(file_path, "wb") as output_file:
            with urllib.request.urlopen(EXTERNAL_DEPENDENCIES[file_path]["url"]) as response:
                length = int(response.info()["Content-Length"])
                counter = 0.0
                MEGABYTES = 2.0 ** 20.0
                while True:
                    data = response.read(8192)
                    if not data:
                        break
                    counter += len(data)
                    output_file.write(data)

                    # We perform animation by overwriting the elements.
                    weights_warning.warning("Downloading %s... (%6.2f/%6.2f MB)" %
                        (file_path, counter / MEGABYTES, length / MEGABYTES))
                    progress_bar.progress(min(counter / length, 1.0))

    # Finally, we remove these visual elements by calling .empty().
    finally:
        if weights_warning is not None:
            weights_warning.empty()
        if progress_bar is not None:
            progress_bar.empty()

# Ensure that load_pg_gan_model is called only once, when the app first loads.
@st.cache(allow_output_mutation=True, hash_funcs=TL_GAN_HASH_FUNCS)
def load_pg_gan_model():
    """
    Create the tensorflow session.
    """
    # Open a new TensorFlow session.
    config = tf.ConfigProto(allow_soft_placement=True)
    session = tf.Session(config=config)

    # Must have a default TensorFlow session established in order to initialize the GAN.
    with session.as_default():
        # Read in either the GPU or the CPU version of the GAN
        with open(MODEL_FILE_GPU if USE_GPU else MODEL_FILE_CPU, 'rb') as f:
            G = pickle.load(f)
    return session, G

# Ensure that load_tl_gan_model is called only once, when the app first loads.
@st.cache(hash_funcs=TL_GAN_HASH_FUNCS)
def load_tl_gan_model():
    """
    Load the linear model (matrix) which maps the feature space
    to the GAN's latent space.
    """
    with open(FEATURE_DIRECTION_FILE, 'rb') as f:
        feature_direction_name = pickle.load(f)

    # Pick apart the feature_direction_name data structure.
    feature_direction = feature_direction_name['direction']
    feature_names = feature_direction_name['name']
    num_feature = feature_direction.shape[1]
    feature_lock_status = np.zeros(num_feature).astype('bool')

    # Rearrange feature directions using Shaobo's library function.
    feature_direction_disentangled = \
        feature_axis.disentangle_feature_axis_by_idx(
            feature_direction,
            idx_base=np.flatnonzero(feature_lock_status))
    return feature_direction_disentangled, feature_names

def get_random_features(feature_names, seed):
    """
    Return a random dictionary from feature names to feature
    values within the range [40,60] (out of [0,100]).
    """
    np.random.seed(seed)
    features = dict((name, 40+np.random.randint(0,21)) for name in feature_names)
    return features

# Hash the TensorFlow session, the pg-GAN model, and the TL-GAN model by id
# to avoid expensive or illegal computations.
@st.cache(show_spinner=False, hash_funcs=TL_GAN_HASH_FUNCS)
def generate_image(session, pg_gan_model, tl_gan_model, features, feature_names):
    """
    Converts a feature vector into an image.
    """
    # Create rescaled feature vector.
    feature_values = np.array([features[name] for name in feature_names])
    feature_values = (feature_values - 50) / 250
    # Multiply by Shaobo's matrix to get the latent variables.
    latents = np.dot(tl_gan_model, feature_values)
    latents = latents.reshape(1, -1)
    dummies = np.zeros([1] + pg_gan_model.input_shapes[1][1:])
    # Feed the latent vector to the GAN in TensorFlow.
    with session.as_default():
        images = pg_gan_model.run(latents, dummies)
    # Rescale and reorient the GAN's output to make an image.
    images = np.clip(np.rint((images + 1.0) / 2.0 * 255.0),
                              0.0, 255.0).astype(np.uint8)  # [-1,1] => [0,255]
    if USE_GPU:
        images = images.transpose(0, 2, 3, 1)  # NCHW => NHWC
    return images[0]

USE_GPU = False
FEATURE_DIRECTION_FILE = "feature_direction_2018102_044444.pkl"
MODEL_FILE_GPU = "karras2018iclr-celebahq-1024x1024-condensed.pkl"
MODEL_FILE_CPU = "karras2018iclr-celebahq-1024x1024-condensed-cpu.pkl"
EXTERNAL_DEPENDENCIES = {
    "feature_direction_2018102_044444.pkl" : {
        "url": "https://streamlit-demo-data.s3-us-west-2.amazonaws.com/facegan/feature_direction_20181002_044444.pkl",
        "size": 164742
    },
    "karras2018iclr-celebahq-1024x1024-condensed.pkl": {
        "url": "https://streamlit-demo-data.s3-us-west-2.amazonaws.com/facegan/karras2018iclr-celebahq-1024x1024-condensed.pkl",
        "size": 92338293
    },
    "karras2018iclr-celebahq-1024x1024-condensed-cpu.pkl": {
        "url": "https://streamlit-demo-data.s3-us-west-2.amazonaws.com/facegan/karras2018iclr-celebahq-1024x1024-condensed-cpu.pkl",
        "size": 92340233
    }
}

if __name__ == "__main__":
    main()


Overwriting score2.py


In [ ]:
!nohup streamlit run score2.py &


nohup: appending output to 'nohup.out'


In [ ]:
from pyngrok import ngrok

url = ngrok.connect(port=8501)
url

'http://e9a99ae03d42.ngrok.io'